# Gep top20 recommendation - XAI recsys: 
## Model: EBM_side_info
## Ranking data: knn_lda_v2

In [1]:
# import caffeine
# caffeine.on(display=False)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Run this for reproduce
# Gets the current working directory
import os
cwd = os.getcwd()
print("Working directory:", cwd)
# Go up one directory from working directory
os.chdir("..")

Working directory: /home/jovyan/1_UT THESIS/CB12_MAIN/nb_recsys_ebm


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Load ranking data (generated by KNN-LDA features)
ranking_data = pd.read_csv('./nb_recsys_ebm/ranking_data_knn_lda_v2.csv')

In [6]:
ranking_data.head()

,UserID,JobID,label,City,State,Country
0,13,828955,0.0,0.0,0.0,1.0
1,13,755985,0.0,0.0,0.0,1.0
2,13,602478,0.0,0.0,0.0,1.0
3,13,299880,0.0,0.0,0.0,1.0
4,13,873212,0.0,0.0,0.0,1.0


In [7]:
features_df = pd.read_csv('./nb_recsys_ebm/features_df_lda.csv')

In [8]:
features_df.head()

,UserID,JobID,City,State,Country,DegreeType,CurrentlyEmployed,ManagedOthers,WorkHistoryTopic,WorkHistoryLevel,SeniorLevel,ReqTopic,DescTopic,TitTopic
0,7,309823,1.0,1.0,1.0,1,1,0,0,1,4.0,18,2,0
1,7,703889,1.0,1.0,1.0,1,1,0,0,1,4.0,18,2,0
2,7,566574,0.0,0.0,1.0,1,1,0,0,1,4.0,0,18,0
3,7,481216,0.0,0.0,1.0,1,1,0,0,1,4.0,18,15,0
4,9,809208,1.0,1.0,1.0,1,1,0,0,1,1.0,5,18,0


In [9]:
# lda jobs features
features_job = ['ReqTopic', 'DescTopic', 'TitTopic']
# lda & user profile features
features_user = ['DegreeType', 'CurrentlyEmployed', 'ManagedOthers', 'WorkHistoryTopic',
       'WorkHistoryLevel', 'SeniorLevel']
# location features
features_location = ['City', 'State', 'Country']

In [10]:
from topN_ebm import *

In [11]:
!cat topN_ebm.py

cat: topN_ebm.py: No such file or directory


# Start topN for a given user_id

- features_job_matrix: feature names for job matrix
- features_work_matrix: feature names for work history matrix
- user_profile_features: list of features for user profile

In [12]:
%%time
import gzip, pickle, pickletools

filepath = "./output_ebm/ebm_side_info_dropid.pikle"

with gzip.open(filepath, 'rb') as f:
    p = pickle.Unpickler(f)
    ebm_side_info = p.load()

CPU times: user 30.2 ms, sys: 34.6 ms, total: 64.8 ms
Wall time: 127 ms


In [13]:
%%time
rec_result_ebm_side_info = topN_ebm_extended(model = ebm_side_info, 
           N = 20, 
           ranking_data = ranking_data, 
           features_df = features_df)

CPU times: user 32min 16s, sys: 772 ms, total: 32min 16s
Wall time: 32min 16s


In [14]:
# Function: get rec_result corresponding to a user id
# rec_result: overall rec_recsult dictionary (output of topN_flat)
def get_rec_result_df(u_id, rec_result):
    rec_20_df = pd.DataFrame(rec_result[u_id])
    rec_cols = ['JobID', 'Y_prob', 'Y_pred']
    rec_20_df.columns = rec_cols
    rec_20_df['UserID'] = u_id
    rec_20_df['rank'] = rec_20_df.groupby('UserID').cumcount()
    return rec_20_df


In [15]:
user_ids = list(ranking_data.groupby('UserID').UserID.unique().astype('int'))

In [16]:
%%time
final_rec_result = pd.DataFrame(columns = ['JobID', 'Y_prob', 'Y_pred','UserID','rank'])

for u_id in user_ids:
    
    temp_df = get_rec_result_df(u_id=u_id, 
                                rec_result=rec_result_ebm_side_info)
    # print(u_id, len(temp_df))
    final_rec_result = pd.concat([final_rec_result, temp_df])

final_rec_result = final_rec_result[['UserID','JobID', 'Y_prob', 'Y_pred', 'rank']]

CPU times: user 9.65 s, sys: 24 ms, total: 9.67 s
Wall time: 9.67 s


In [17]:
final_rec_result.head()

,UserID,JobID,Y_prob,Y_pred,rank
0,13,32115,0.665981,1,0
1,13,1081720,0.665981,1,1
2,13,494993,0.629645,1,2
3,13,755985,0.611904,1,3
4,13,602478,0.611904,1,4


In [18]:
final_rec_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 73820 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   UserID  73820 non-null  object 
 1   JobID   73820 non-null  object 
 2   Y_prob  73820 non-null  float64
 3   Y_pred  73820 non-null  object 
 4   rank    73820 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.4+ MB


In [19]:
final_rec_result.to_csv('./output_topN_ebm/rec_result_side_info_knn.csv', header=True, index=False)